In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

import librosa
import os as os
from scipy.misc import comb
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm
from keras.models import load_model

C:\Users\Blaine\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_PATH = "./large_files/audio/"  


def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)
# Get available labels
labels, indices, _ = get_labels(DATA_PATH)


# Getting first arrays
X = np.load(labels[0] + '.npy')
y = np.zeros(X.shape[0])

# Append all of the dataset into one single array, same goes for y
for i, label in enumerate(labels[1:]):
    x = np.load(label + '.npy')
    X = np.vstack((X, x))
    y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

assert X.shape[0] == len(y)

In [3]:
%reload_ext autoreload
%autoreload 2

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


# Second dimension of the feature is dim2
feature_dim_2 = 11

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 20
batch_size = 50
verbose = 2
num_classes = 30

# Reshaping to perform 2D convolution
X = X.reshape(X.shape[0], feature_dim_1, feature_dim_2, channel)


y_hot = to_categorical(y)

In [4]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

def get_larger_model():
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(96, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(240, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

NameError: name 'numpy' is not defined

In [ ]:
# k=2
# num_val_samples = len(X) // k
# all_scores = []


# for i in range(k):
#     print('processing fold #', i)
#     # Prepare the validation data: data from partition # k
#     val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
#     val_targets = y_hot[i * num_val_samples: (i + 1) * num_val_samples]

#     # Prepare the training data: data from all other partitions
#     partial_train_data = np.concatenate(
#         [X[:i * num_val_samples],
#          X[(i + 1) * num_val_samples:]],
#         axis=0)
#     partial_train_targets = np.concatenate(
#         [y_hot[:i * num_val_samples],
#          y_hot[(i + 1) * num_val_samples:]],
#         axis=0)

model = get_model()
model.fit(X, y_hot,
batch_size=batch_size, 
epochs=epochs, 
verbose=verbose         )
#     val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
#     all_scores.append(val_mae)

In [5]:
model = get_larger_model()
model.fit(X, y_hot,
batch_size=batch_size, 
epochs=epochs, 
verbose=verbose)

Epoch 1/20
 - 17s - loss: 2.4936 - acc: 0.2735
Epoch 2/20
 - 10s - loss: 1.5122 - acc: 0.5557
Epoch 3/20
 - 10s - loss: 1.2750 - acc: 0.6302
Epoch 4/20
 - 10s - loss: 1.1555 - acc: 0.6642
Epoch 5/20
 - 10s - loss: 1.0790 - acc: 0.6862
Epoch 6/20
 - 10s - loss: 1.0377 - acc: 0.6979
Epoch 7/20
 - 10s - loss: 0.9899 - acc: 0.7138
Epoch 8/20
 - 10s - loss: 0.9640 - acc: 0.7214
Epoch 9/20
 - 10s - loss: 0.9412 - acc: 0.7312
Epoch 10/20
 - 10s - loss: 0.9088 - acc: 0.7394
Epoch 11/20
 - 10s - loss: 0.9010 - acc: 0.7434
Epoch 12/20
 - 10s - loss: 0.8848 - acc: 0.7474
Epoch 13/20
 - 10s - loss: 0.8694 - acc: 0.7509
Epoch 14/20
 - 11s - loss: 0.8559 - acc: 0.7560
Epoch 15/20
 - 10s - loss: 0.8517 - acc: 0.7583
Epoch 16/20
 - 10s - loss: 0.8545 - acc: 0.7567
Epoch 17/20
 - 10s - loss: 0.8487 - acc: 0.7603
Epoch 18/20
 - 11s - loss: 0.8426 - acc: 0.7634
Epoch 19/20
 - 11s - loss: 0.8376 - acc: 0.7628
Epoch 20/20
 - 11s - loss: 0.8362 - acc: 0.7651


In [ ]:
model.save('good_model.h5')